In [ ]:
# Autoencoder neural network for dimensionality reduction of rehabilitation movement data

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility


# It is strongly recommended to use a GPU for this step, otherwise the training may take a very long time
# If you do use a CPU, replace CuDDLSTM with LSTM below, and also in cell 5
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, LSTM
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
now = datetime.datetime.now

batch_size = 10

# length of data sequences
n_timesteps = 240
# dimension of data sequences
n_dim = 117

In [ ]:
# Import the data
import M01ViconDataLoader
X_correct, X_incorrect = M01ViconDataLoader.load_data()

print(X_correct.shape, 'correct sequences')
print(X_incorrect.shape, 'incorrect sequences')

# Plot the first sequences of correct and incorrect data
plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
plt.plot(X_correct[0])
plt.ylim([-1,1])
plt.subplot(1,2,2)
plt.plot(X_incorrect[0])
plt.ylim([-1,1])
plt.tight_layout()
plt.show()

In [ ]:
# Add 50 time frames at the beginning and end of sequences
# The autoencoder has difficulties with the beginning and ending frames
data_correct = np.zeros((X_correct.shape[0],n_timesteps+100,n_dim))
for i in range(X_correct.shape[0]):
    data_correct[i,:,:] = np.concatenate((np.concatenate((np.tile(X_correct[i,0,:],[50, 1]), X_correct[i,:,:])), np.tile(X_correct[i,-1,:],[50, 1])))

data_incorrect = np.zeros((X_incorrect.shape[0],n_timesteps+100,n_dim))
for i in range(X_incorrect.shape[0]):
    data_incorrect[i,:,:] = np.concatenate((np.concatenate((np.tile(X_incorrect[i,0,:],[50, 1]), X_incorrect[i,:,:])), np.tile(X_incorrect[i,-1,:],[50, 1])))
    
# Plot the first sequences of correct and incorrect data
plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
plt.plot(data_correct[0])
plt.ylim([-1,1])
plt.subplot(1,2,2)
plt.plot(data_incorrect[0])
plt.ylim([-1,1])
plt.tight_layout()
plt.show()

In [ ]:
# Encoder layers
input_seq = Input(shape=(n_timesteps+100,n_dim))
encoded1 = LSTM(30,return_sequences = True)(input_seq)
encoded2 = LSTM(10,return_sequences = True)(encoded1)
# Encoded representation of the input, 340x4 vector
encoded = LSTM(4,return_sequences = True)(encoded2)
# Decoder layers
decoded1 = LSTM(10,return_sequences = True)(encoded)
decoded2 = LSTM(30,return_sequences = True)(decoded1)
decoded = LSTM(n_dim, return_sequences = True)(decoded2)

# The model maps an input to its reconstruction
autoencoder = Model(inputs=input_seq, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
# Shuffle the sequences to create a training and validation sets
import random
trainidx = random.sample(range(0,data_correct.shape[0]),63)
valididx = np.setdiff1d(np.arange(0,90,1),trainidx)
train_data = data_correct[trainidx,:,:]
valid_data = data_correct[valididx,:,:]

In [ ]:
# Train an autoencoder on the correct data sequences

# Measure the training time 
t = now()

# Request to stop before reaching the number of epochs if the validation loss does not decrease for 1000 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience = 1000)

history = autoencoder.fit(train_data, train_data, epochs = 10000, batch_size = batch_size, shuffle=True,
                validation_data=(valid_data, valid_data), verbose = 0, callbacks = [early_stopping])

print('Training time: %s' % (now() - t))

In [ ]:
# Plot the training and validation results
plt.figure(1)
plt.subplot(121)
plt.plot(history.history['loss'])
plt.title('Loss')
plt.subplot(122)
plt.plot(history.history['val_loss'])
plt.title('Validation Loss')
plt.tight_layout()
plt.show()

# Print the resulting training and validation loss values
print(history.history['loss'][-1])
print(history.history['val_loss'][-1])

In [ ]:
# Encode and decode sequences to check the model performance
decoded_seqs = autoencoder.predict(data_correct)
# Plot the results
n = 2  # how many sequences we will display
plt.figure(figsize = (12,6))
for i in range(n):
    # display original sequences
    plt.subplot(n, 2, 2*i+1)
    plt.plot(data_correct[i])
    
    # display reconstruction
    plt.subplot(n, 2, 2*i+2)
    plt.plot(decoded_seqs[i])
plt.tight_layout()
plt.show()

In [ ]:
# Create an encoder model, that maps an input to its encoded representation
encoder = Model(inputs=input_seq, outputs=encoded)

# Test the encoder model
encoded_seqs = encoder.predict(data_correct)

# Plot the results
n = 2  # how many sequences we will display
plt.figure(figsize = (12,6))
for i in range(n):
    # display original sequences
    plt.subplot(n, 2, 2*i+1)
    plt.plot(data_correct[i])
    
    # display reconstruction
    plt.subplot(n, 2, 2*i+2)
    plt.plot(encoded_seqs[i])
plt.tight_layout()
plt.show()

In [ ]:
# Plot all encoded sequences without the first and last 50 time frames
plt.figure(figsize = (14,14))
for i in range(data_correct.shape[0]):
    plt.subplot(4,1,1)
    plt.plot(encoded_seqs[i,50:-50,0])
    plt.xlabel('Time Frame',fontsize=12)
    plt.ylabel('Angle (Degrees)',fontsize=12)
    plt.subplot(4,1,2)
    plt.plot(encoded_seqs[i,50:-50,1])
    plt.xlabel('Time Frame',fontsize=12)
    plt.ylabel('Angle (Degrees)',fontsize=12)
    plt.subplot(4,1,3)
    plt.plot(encoded_seqs[i,50:-50,2])
    plt.xlabel('Time Frame',fontsize=12)
    plt.ylabel('Angle (Degrees)',fontsize=12)
    plt.subplot(4,1,4)
    plt.plot(encoded_seqs[i,50:-50,3])
    plt.xlabel('Time Frame',fontsize=12)
    plt.ylabel('Angle (Degrees)',fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# autoencoder.save_weights('M01_Autoencoder.h5')

In [ ]:
# Remove the added first and last 50 frames 
encoded_seqs = encoded_seqs[:,50:-50,:]

print(encoded_seqs.shape, 'encoded sequences shape')
# Reshape the encoded sequences, because savetxt saves two dimensional data
seqs = encoded_seqs.reshape(encoded_seqs.shape[0],encoded_seqs.shape[1]*encoded_seqs.shape[2])
print(seqs.shape, 'encoded sequences shape for saving')
# Save the data in the file 'Autoencoder_Output_Correct.csv'
np.savetxt('Autoencoder_Output_Correct.csv', seqs, fmt='%.5f',delimiter=',')

In [ ]:
# Reduce the dimensionality of the incorrect sequences
encoded_seqs_incorrect = encoder.predict(data_incorrect)

In [ ]:
# Remove the added first and last 50 frames 
encoded_seqs_incorrect = encoded_seqs_incorrect[:,50:-50,:]

print(encoded_seqs_incorrect.shape, 'encoded incorrect sequences shape')
# Reshape the encoded sequences, because savetxt saves only tow dimensional data
seqs_incorrect = encoded_seqs_incorrect.reshape(encoded_seqs_incorrect.shape[0],encoded_seqs_incorrect.shape[1]*encoded_seqs_incorrect.shape[2])
print(seqs_incorrect.shape, 'encoded incorrect sequences shape for saving')
# Save the incorrect data in the file 'Autoencoder_Output_Incorrect.csv'
np.savetxt('Autoencoder_Output_Incorrect.csv', seqs_incorrect, fmt='%.5f',delimiter=',')